In [1]:
#Importando librerías
from sympy import *
from sympy.parsing.sympy_parser import *
from sympy.physics.qho_1d import *
from matplotlib import *
from ipywidgets import *

#Impresión para "jupyter notebook"
init_printing(use_latex = 'mathjax')

In [2]:
[n, p, x, m, l, hbar, omega, alpha] = symbols('n p x m l hbar omega, alpha', real = True, positive = True)

In [3]:
items_layout = Layout(width='auto')     # override the default width of the button to 'auto' to let the button grow

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')
Title = Label(value = "El programa ya reconoce las variables simbolicas $n$, $l$, $p$, $x$, $m$, $hbar$ y $omega$.")
Problem = RadioButtons(options = ['Harmonic Oscilator', 'Particle In a Box'], layout = items_layout)
Function = Checkbox(description='\(¿Calculate Psi_n? \)')
Energy = Checkbox(description='\(¿Calculate E_n? \)')
Variables = Text(description = '\(Vars= \)', layout = items_layout)
Perturbation = Text(description = '\(H^´= \)', layout = items_layout)
Order = IntSlider(min = 0, max = 2, description = '\(ECO: \)', layout = items_layout)
Level = BoundedIntText(min = 0, description = '\(n= \)', layout = items_layout)
Iterations = BoundedIntText(min = 1, description = '\(Iter= \)', layout = items_layout)

items = [Function, Energy, Variables, Perturbation, Order, Level, Iterations]

box = Box(children=items, layout=box_layout)

display(Title)
display(Problem)
display(box)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [4]:
var(Variables.value)
Hp = eval(Perturbation.value)
O = Order.value
n = Level.value
p = Iterations.value

In [5]:
def phi(n):
    if Problem.value == 'Harmonic Oscilator':
        return psi_n(n, x, m, omega)
    if Problem.value == 'Particle In a Box':
        return sqrt(2/l)*sin(((n*pi)/l)*x)

In [6]:
def E(n):
    if Problem.value == 'Harmonic Oscilator':
        return E_n(n, omega)
    if Problem.value == 'Particle In a Box':
        return (n**2*pi**2*hbar**2)/(2*m*l**2)

In [7]:
if Energy.value == True:
    EO0 = E(n)
    Emat = Matrix([EO0])
    if O >= 1:
        IntegrandEO1 = phi(n)*Hp*phi(n)
        IntEO1 = integrate(IntegrandEO1, (x, -oo, oo))
        EO1 = IntEO1
        Emat = Emat.row_insert(1, Matrix([EO1]))
    if O == 2:
        for i in range(0, p+1):
            j = i
            if i != n:
                if ((n == 0) and (i == 1)) or (i==0):
                    IntegrandEO2 = Matrix([phi(i)*Hp*phi(n)])
                    DenominatorEO2 = Matrix([E(n)-E(i)])
                else:
                    IntegrandEO2 = IntegrandEO2.row_insert(j-1, Matrix([phi(i)*Hp*phi(n)]))
                    DenominatorEO2 = DenominatorEO2.row_insert(j-1, Matrix([E(n)-E(i)]))
        IntEO2 = integrate(IntegrandEO2, (x, -oo, oo))
        IntEO2Sq = zeros(len(IntEO2), 1)
        for i in range(p):
            IntEO2Sq[i] = IntEO2[i]**2
        DivisionEO2 = zeros(p, 1)
        for i in range(p):
            DivisionEO2[i] = IntEO2Sq[i]/DenominatorEO2[i]
        EO2 = 0
        for i in range(p):
            EO2 = EO2 + DivisionEO2[i]
        Emat = Emat.row_insert(2, Matrix([EO2]))
    Ep = 0
    for i in range(O+1):
        Ep = Ep + Emat[i]
if Function.value == True:
    PHIO0 = phi(n)
    PHImat = Matrix([PHIO0])
    for i in range(0, p+1):
        j = i
        if i != n:
            if ((n == 0) and (i == 1)) or (i==0):
                Phi_pO1 = Matrix([phi(i)])
            else:
                Phi_pO1 = Phi_pO1.row_insert(j-1, Matrix([phi(i)]))
    ProductPHIO1 = zeros(p, 1)
    for i in range(p):
        ProductPHIO1[i] = (IntEO2[i]/DenominatorEO2[i])*Phi_pO1[i]
    PHIO1 = 0
    for i in range(p):
        PHIO1 = PHIO1 + ProductPHIO1[i]
    PHImat = PHImat.row_insert(1, Matrix([PHIO1]))
    PHIp = 0
    for i in range(2):
        PHIp = PHIp + PHImat[i]

In [11]:
Emat

⎡        ℏ⋅ω         ⎤
⎢        ───         ⎥
⎢         2          ⎥
⎢                    ⎥
⎢          2         ⎥
⎢       3⋅ℏ ⋅d       ⎥
⎢      ───────       ⎥
⎢         2  2       ⎥
⎢      4⋅m ⋅ω        ⎥
⎢                    ⎥
⎢      2  2      3  2⎥
⎢  11⋅ℏ ⋅c    9⋅ℏ ⋅d ⎥
⎢- ──────── - ───────⎥
⎢     3  4       4  5⎥
⎣  8⋅m ⋅ω     4⋅m ⋅ω ⎦

In [12]:
Ep

      2  2      3  2       2        
  11⋅ℏ ⋅c    9⋅ℏ ⋅d     3⋅ℏ ⋅d   ℏ⋅ω
- ──────── - ─────── + ─────── + ───
     3  4       4  5      2  2    2 
  8⋅m ⋅ω     4⋅m ⋅ω    4⋅m ⋅ω       

In [13]:
PHImat

NameError: name 'PHImat' is not defined

In [14]:
PHIp

NameError: name 'PHIp' is not defined

## Teoría de perturbaciones

Consiste en resolver un sistema perturbado(se conoce la solución al no perturbado), y donde el interés es conocer la contribución de la parte perturbada $H'$ al nuevo sistema total. 

$$ H = H^{0} + H'$$

Para sistemas no degenerados, la corrección a la energía a primer orden se calcula como 

$$ E_{n}^{(1)} = \int\psi_{n}^{(0)*} H' \psi_{n}^{(0)}d\tau$$

** Tarea 1 : Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

El programa ya reconoce las variables simbolicas x, m y hbar.

¿Desea declarar más variables? (0:No; 1:Sí)
1

Añada las variables simbólicas que requiera. Cuando haya finalizado, ingrese 'done'.
var_1 = alpha, real = True
var_2 = pi
var_3 = done


Y la corrección a la función de onda, también a primer orden, se obtiene como:

 $$ \psi_{n}^{(1)} = \sum_{m\neq n} \frac{\langle\psi_{m}^{(0)} | H' | \psi_{n}^{(0)} \rangle}{E_{n}^{(0)} - E_{m}^{(0)}} \psi_{m}^{(0)}$$

**Tarea 2: Programar esta ecuación si conoces $H^{0}$ y sus soluciones. **

In [ ]:
### Solución

**Tarea 3: Investigue las soluciones a segundo orden y también programe las soluciones. **

In [ ]:
### Solución




**Tarea 4.  Resolver el átomo de helio aplicando los programas anteriores.** 

**Tarea 5: Método variacional-perturbativo. **

Este método nos permite estimar de forma precisa $E^{(2)}$ y correcciones perturbativas de la energía de órdenes más elevados para el estado fundamental del sistema, sin evaluar sumas infinitas. Ver ecuación 9.38 del libro. 

**Resolver el átomo de helio, considerando este método (sección 9.4), como mejor le parezca. **

**Tarea 6. Revisar sección 9.7. **

Inicialmente a mano, y sengunda instancia favor de intentar programar sección del problema, i.e. integral de Coulomb  e integral de intercambio.

## Siguiente: Segunda parte, Octubre
Simetrías moleculares y Hartree-Fock